# CSV a txts

Vamos a generar los inserts para las tablas de Autosummit Perú SAC

Importar librerías

In [19]:
import pandas as pd
import numpy as np

Abrir CSV

In [20]:
ds_ASP = pd.read_csv('ASP.csv', sep=',')

Ejemplos

In [21]:
ds_ASP['Cliente']

0                (42206340) DANIELLA MARIA BOLAÑOS GAMERO
1                         (20100115663) PANDERO S.A. EAFC
2       (09468059) KATIA NATHALI DE LOAYZA WONG DE PAC...
3        (46472213) JHONATHAN MITCHELL ANTEZANA ESCALANTE
4                (42607724) KRISCIA ZULAY REATEGUI ZAMORA
                              ...                        
1253                                                  NaN
1254                                                  NaN
1255                                                  NaN
1256                                                  NaN
1257                                                  NaN
Name: Cliente, Length: 1258, dtype: object

<p>Filtrado de colores</p>

In [22]:
colors_rep = ds_ASP['Color']
colors = []

for c in colors_rep:
    if isinstance(c, str) and c.strip() != '' and c.lower() != 'nan':
        if c not in colors:
            colors.append(c)

colors = sorted(colors)

text_colors = '-- migrate:up\n\n'
id = 1

for c in colors:
    c_escaped = c.replace("'", "''")
    text_colors += f"INSERT INTO colors (id, nombre) VALUES ({id}, '{c_escaped}');\n"
    id += 1

text_colors += '\n-- migrate:down\nDELETE FROM colors;'
with open('inserts_colors.sql', 'w') as f:
    f.write(text_colors)


<p>Filtrado por Asesor </p>

In [23]:
import re
asesores_rep = ds_ASP['Asesor']
asesores = []

#En caso tenga nombres "raros" el asesor
PALABRAS_INVALIDAS = {
    "PDI", "EXHIBICION", "ATE", "CASO", "ENTREGA"
}

#Almacenar todo en diccionario - para evitar duplicados
asesores = {} 

def es_asesor_valido(texto):
    if not isinstance(texto, str):
        return False

    texto = texto.strip().upper()

    # Debe tener al menos un espacio (nombre + apellido)
    if " " not in texto:
        return False

    # No debe contener comas ni números
    if "," in texto or re.search(r"\d", texto):
        return False

    # No debe contener palabras inválidas
    for palabra in PALABRAS_INVALIDAS:
        if palabra in texto:
            return False

    # Solo letras y espacios
    if not re.match(r"^[A-ZÁÉÍÓÚÑ ]+$", texto):
        return False

    return True


for c in asesores_rep:
    if not es_asesor_valido(c):
        continue

    c = c.strip().upper()
    nombres, apellidos = c.split(" ", 1)

    key = (nombres, apellidos)
    asesores[key] = True

sql_asesores = "-- migrate:up\n\n"
id_asesor = 1

for nombres, apellidos in sorted(asesores.keys()):
    sql_asesores += (
        "INSERT INTO asesores (id, nombres, apellidos) "
        f"VALUES ({id_asesor}, '{nombres}', '{apellidos}');\n"
    )
    id_asesor += 1

sql_asesores += "\n-- migrate:down\nDELETE FROM asesores;\n"

with open("inserts_asesores.sql", "w", encoding="utf-8") as f:
    f.write(sql_asesores)


<h3>Descomposición en marca (nombre) y modelo (nombre, version, año)</h3>

<p> Considerar que Marca no está de forma explícita en el CSV. Solución: inferir y filtrar con diccionarios </p>

In [24]:
import re

# =========================
# 1. COLUMNA CSV
# =========================
modelos_rep = ds_ASP['Modelo']

# Diccionarios (evitan duplicados automáticamente)
marcas = {}      # { "FORD": 1 }
modelos = {}     # { (modelo_base, version, traccion, anio, motor, marca): True }

# =========================
# CATÁLOGOS CONTROLADOS
# =========================
MAPA_MARCAS = {
    "FORD": [
        "TERRITORY", "RANGER", "F-150", "MAVERICK",
        "EXPLORER", "ESCAPE", "BRONCO",
        "MUSTANG", "EXPEDITION"
    ],
    "CHERY": [
        "TIGGO", "ARRIZO", "M7", "HIMLA"
    ]
}

VERSIONES_VALIDAS = {
    "TITANIUM", "TREND", "XLS", "XLT", "XL", "LTD",
    "PLATINUM", "RAPTOR", "BADLANDS", "LARIAT",
    "ACTIVE", "ST", "PREMIUM", "PRO", "MAX",
    "BIG", "BEND", "TREMOR", "GT"
}

TRACCIONES = {"4X2", "4X4", "AWD", "4WD"}
DESCARTES = {"MT", "AT", "CVT", "DCT", "FHEV", "MHEV", "PHEV", "GLP", "GNV"}

# =========================
# FUNCIONES
# =========================

def detectar_marca(texto: str):
    texto = texto.upper()
    for marca, modelos in MAPA_MARCAS.items():
        for m in modelos:
            if m in texto:
                return marca
    return None


def extraer_anio(texto: str):
    match = re.search(r"(20\d{2})", texto)
    return int(match.group(1)) if match else None


def separar_modelo(texto: str):
    tokens = texto.upper().split()

    motor = None
    traccion = None
    usados = set()

    for t in tokens:
        # Motor (1.5L, 2.0T, etc.)
        if re.match(r"\d\.\d(T|L)?", t):
            motor = float(re.findall(r"\d\.\d", t)[0])
            usados.add(t)

        # Tracción
        elif t in TRACCIONES:
            traccion = t
            usados.add(t)

        # Tokens técnicos descartables
        elif t in DESCARTES:
            usados.add(t)

    # limpiar tokens ya usados
    limpio = [t for t in tokens if t not in usados]

    modelo_base = limpio[0]
    version_tokens = [t for t in limpio[1:] if t in VERSIONES_VALIDAS]
    version_modelo = " ".join(version_tokens)

    return modelo_base, version_modelo, traccion, motor

# =========================
# PROCESAMIENTO DEL CSV
# =========================

for fila in modelos_rep:
    if not isinstance(fila, str) or fila.strip() == "":
        continue

    fila = fila.strip().upper()

    marca = detectar_marca(fila)
    anio = extraer_anio(fila)

    if not marca or not anio:
        continue

    modelo_base, version, traccion, motor = separar_modelo(fila)

    if not all([modelo_base, version, traccion, motor]):
        continue

    # registrar marca (PK lógica)
    if marca not in marcas:
        marcas[marca] = len(marcas) + 1

    key = (modelo_base, version, traccion, anio, motor, marca)
    modelos[key] = True

# =========================
# SQL: MARCAS
# =========================

sql_marcas = "-- migrate:up\n\n"

for marca, id_marca in marcas.items():
    sql_marcas += (
        f"INSERT INTO marcas (id_marca, nombre) "
        f"VALUES ({id_marca}, '{marca}');\n"
    )

sql_marcas += "\n-- migrate:down\nDELETE FROM marcas;\n"

with open("inserts_marcas.sql", "w", encoding="utf-8") as f:
    f.write(sql_marcas)

# =========================
# SQL: MODELOS
# =========================

sql_modelos = "-- migrate:up\n\n"
id_modelo = 1

for (modelo_base, version, traccion, anio, motor, marca) in modelos.keys():
    marca_id = marcas[marca]

    sql_modelos += (
        "INSERT INTO modelos "
        "(id_modelo, modelo_base, version_modelo, traccion, anio, motor, marca_id) "
        f"VALUES ({id_modelo}, '{modelo_base}', '{version}', "
        f"'{traccion}', {anio}, {motor}, {marca_id});\n"
    )
    id_modelo += 1

sql_modelos += "\n-- migrate:down\nDELETE FROM modelos;\n"

with open("inserts_modelos.sql", "w", encoding="utf-8") as f:
    f.write(sql_modelos)


<h2>Filtrado de GPS</h2>
<p>Eliminar si tiene la palabra "GPS" para dejar el resto del texto </p>

In [25]:
import pandas as pd
import re

def limpiar_gps(texto):
    if not isinstance(texto, str):
        return None

    texto = texto.strip().upper()

    # Eliminar fechas entre paréntesis
    texto = re.sub(r"\(.*?\)", "", texto)

    return texto.strip()

ds = pd.read_csv("ASP.csv", encoding="utf-8")

gps_dict = {}
id_gps = 1

for g in ds['GPS']:
    nombre = limpiar_gps(g)
    if not nombre:
        continue

    if nombre not in gps_dict:
        gps_dict[nombre] = id_gps
        id_gps += 1

# Generar SQL GPS

sql_gps = "-- migrate:up\n\n"

for nombre, id_ in gps_dict.items():
    sql_gps += (
        "INSERT INTO gps (id, nombre) "
        f"VALUES ({id_}, '{nombre}');\n"
    )

sql_gps += "\n-- migrate:down\nDELETE FROM gps;\n"

with open("inserts_gps.sql", "w", encoding="utf-8") as f:
    f.write(sql_gps)


<h2>Filtrado de Clientes </h2>
<p>Al estar en cliente el nombre de empresa / persona natural + DNI/RUC El objetivos principal será separar número de identificación de identidad y el nombre, así como clasificar en caso sea persona natural o empresa </p>

In [26]:
import re

clientes_rep = ds_ASP['Cliente']

# ===================== FUNCIONES =====================

def limpiar_cliente(texto):
    """
    (20100115663) PANDERO S.A. EAFC
    """
    if not isinstance(texto, str):
        return None, None

    texto = texto.strip()

    match = re.match(r"\((\d+)\)\s*(.+)", texto)
    if not match:
        return None, None

    numero = match.group(1)
    nombre = match.group(2).strip().upper()

    return numero, nombre


PALABRAS_EMPRESA = {
    "S.A", "S.A.", "S.A.C", "SAC", "SOCIEDAD",
    "EMPRESA", "E.A.F.C", "EAFC", "CORPORACION",
    "GRUPO", "GROUP", "E.I.R.L", "SERVICIOS",
    "SRL", "S.R.L"
}

def es_empresa(nombre):
    return any(p in nombre for p in PALABRAS_EMPRESA)


# ===================== ESTRUCTURAS =====================

clientes = {}          # id_cliente -> (numero, nombre)
personas = set()       # ids
empresas = set()       # ids

id_cliente = 1

# ===================== PROCESAMIENTO =====================

for fila in clientes_rep:
    numero, nombre = limpiar_cliente(fila)

    if not numero or not nombre:
        continue

    clientes[id_cliente] = (numero, nombre)

    if es_empresa(nombre):
        empresas.add(id_cliente)
    else:
        personas.add(id_cliente)

    id_cliente += 1


# ===================== SQL CLIENTES =====================

sql_clientes = "-- migrate:up\n\n"

for id_cliente, (numero, nombre) in clientes.items():
    nombre = nombre.replace("'", "''")
    sql_clientes += (
        "INSERT INTO clientes (id_cliente, Numero_Identificacion, nombre) "
        f"VALUES ({id_cliente}, '{numero}', '{nombre}');\n"
    )

sql_clientes += "\n-- migrate:down\nDELETE FROM clientes;\n"

with open("inserts_clientes.sql", "w", encoding="utf-8") as f:
    f.write(sql_clientes)


# ===================== SQL PERSONA NATURAL =====================

sql_personas = "-- migrate:up\n\n"

for id_cliente in personas:
    sql_personas += (
        "INSERT INTO persona_natural (id_persona, cliente_id) "
        f"VALUES ({id_cliente}, {id_cliente});\n"
    )

sql_personas += "\n-- migrate:down\nDELETE FROM persona_natural;\n"

with open("inserts_persona_natural.sql", "w", encoding="utf-8") as f:
    f.write(sql_personas)


# ===================== SQL EMPRESA =====================

sql_empresas = "-- migrate:up\n\n"

for id_cliente in empresas:
    sql_empresas += (
        "INSERT INTO empresa (id_empresa, cliente_id) "
        f"VALUES ({id_cliente}, {id_cliente});\n"
    )

sql_empresas += "\n-- migrate:down\nDELETE FROM empresa;\n"

with open("inserts_empresa.sql", "w", encoding="utf-8") as f:
    f.write(sql_empresas)


Filtrado vehiculo

In [27]:
# ===============================
# VEHICULOS
# ===============================

sql_vehiculos = "-- migrate:up\n\n"

for _, row in ds.iterrows():

    placa = str(row['PLACA']).strip()
    vin = str(row['VIN']).strip()

    color = str(row['Color']).strip().upper()
    modelo = str(row['Modelo']).strip().upper()

    dni, _ = limpiar_cliente(row['Cliente'])
    gps = limpiar_gps(row['GPS'])

    if not (placa and vin and color and modelo and dni):
        continue

    color_id = colores[color]
    modelo_id = modelos[modelo]
    cliente_id = dict_cliente[dni]
    gps_id = gps_dict.get(gps, "NULL")

    sql_vehiculos += (
        "INSERT INTO vehiculos "
        "(placa, vin, color_id, modelo_id, cliente_id, gps_id) "
        f"VALUES ('{placa}', '{vin}', {color_id}, {modelo_id}, {cliente_id}, {gps_id});\n"
    )

sql_vehiculos += "\n-- migrate:down\nDELETE FROM vehiculos;\n"

with open("inserts_vehiculos.sql", "w", encoding="utf-8") as f:
    f.write(sql_vehiculos)


NameError: name 'colores' is not defined

Filtrado de recepciones_campañas

In [ ]:
import pandas as pd

# ===============================
# Cargar dataset
# ===============================

ds = pd.read_csv("ASP.csv", encoding="utf-8")

# ===============================
# CAMPAÑAS
# ===============================

campanias = {}
id_campania = 1

for c in ds['CAMPAÑA']:
    if not isinstance(c, str):
        continue

    c = c.strip().upper()

    if c and c not in campanias:
        campanias[c] = id_campania
        id_campania += 1

# ===============================
# RECEPCIONES
# ===============================

recepciones = {}
id_recepcion = 1

for f in ds['FECHA DE RECEPCION DEL VEHICULO']:
    if pd.isna(f):
        continue

    f = str(f).strip()

    if f not in recepciones:
        recepciones[f] = id_recepcion
        id_recepcion += 1

# ===============================
# RELACION N:M
# ===============================

relaciones = set()

for _, row in ds.iterrows():

    camp = str(row['CAMPAÑA']).strip().upper()
    fecha = str(row['FECHA DE RECEPCION DEL VEHICULO']).strip()

    if not camp or camp == "NAN" or not fecha or fecha == "NAN":
        continue

    relaciones.add((campanias[camp], recepciones[fecha]))

# ===============================
# GENERAR SQL
# ===============================

sql_rel = "-- migrate:up\n\n"

for camp_id, rec_id in sorted(relaciones):
    sql_rel += (
        "INSERT INTO recepciones_campañas (campañas_id, recepciones_id) "
        f"VALUES ({camp_id}, {rec_id});\n"
    )

sql_rel += "\n-- migrate:down\nDELETE FROM recepciones_campañas;\n"

with open("inserts_recepciones_campañas.sql", "w", encoding="utf-8") as f:
    f.write(sql_rel)


Filtrado de ubicaciones

In [ ]:
import pandas as pd

# ===============================
# Cargar dataset
# ===============================

ds = pd.read_csv("ASP.csv", encoding="utf-8")

# ===============================
# LIMPIEZA
# ===============================

def limpiar_texto(txt):
    if not isinstance(txt, str):
        return None
    return txt.strip().upper()

# ===============================
# DISTRITOS (CATALOGO)
# ===============================

distritos = {}
id_distrito = 1

for u in ds['UBICACIÓN']:
    u = limpiar_texto(u)
    if not u:
        continue

    if u not in distritos:
        distritos[u] = id_distrito
        id_distrito += 1

# ===============================
# UBICACIONES
# ===============================

ubicaciones = {}

for u in ds['UBICACIÓN']:
    u = limpiar_texto(u)
    if not u:
        continue

    ubicaciones[u] = distritos[u]

# ===============================
# SQL DISTRITOS
# ===============================

sql_distritos = "-- migrate:up\n\n"

for nombre, id_ in distritos.items():
    sql_distritos += (
        "INSERT INTO distritos (id, nombre) "
        f"VALUES ({id_}, '{nombre}');\n"
    )

sql_distritos += "\n-- migrate:down\nDELETE FROM distritos;\n"

with open("inserts_distritos.sql", "w", encoding="utf-8") as f:
    f.write(sql_distritos)

# ===============================
# SQL UBICACIONES
# ===============================

sql_ubicaciones = "-- migrate:up\n\n"

for ubicacion, distrito_id in ubicaciones.items():
    sql_ubicaciones += (
        "INSERT INTO ubicaciones (ubicacion, distritos_id) "
        f"VALUES ('{ubicacion}', {distrito_id});\n"
    )

sql_ubicaciones += "\n-- migrate:down\nDELETE FROM ubicaciones;\n"

with open("inserts_ubicaciones.sql", "w", encoding="utf-8") as f:
    f.write(sql_ubicaciones)
